# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [5]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [6]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv(r'C:\Users\Gamer PC\Desktop\python_api_challenge\Starter_Code\output_data\cities.csv')

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,zoucheng,35.4006,116.9656,62.76,25,0,4.21,CN,1715528213
1,1,moyamba,8.1667,-12.4333,90.05,61,99,4.85,SL,1715528213
2,2,fort bragg,35.1390,-79.0060,76.82,34,0,11.50,US,1715528165
3,3,crane,31.3974,-102.3501,73.83,65,100,6.49,US,1715528172
4,4,trinity beach,-16.7888,145.6968,70.02,93,10,10.36,AU,1715528213


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                      color='Humidity', cmap='blues', hover_cols=['City', 'Humidity'],
                                      tiles='OSM', frame_width=700, frame_height=500,
                                      title='Humidity in Various Cities')

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [9]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_cities_df = city_data_df[(city_data_df['Max Temp'] > 70) & 
                               (city_data_df['Max Temp'] < 80) & 
                               (city_data_df['Humidity'] < 70) & 
                               (city_data_df['Wind Speed'] < 10) & 
                               (city_data_df['Cloudiness'] < 20)]

# Drop any rows with null values
ideal_cities_df = ideal_cities_df.dropna()

# Display sample data
ideal_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
32,32,villa de arista,22.6500,-100.8333,79.61,23,1,7.00,MX,1715528216
40,40,hamilton,39.1834,-84.5333,70.97,46,0,4.00,US,1715527881
73,73,greenville,35.6127,-77.3663,74.12,40,0,9.22,US,1715527920
184,184,bloemfontein,-29.1211,26.2140,74.68,12,0,5.75,ZA,1715528151
364,364,covington,39.0837,-84.5086,70.86,47,0,1.99,US,1715527676
424,424,heroica caborca,30.7167,-112.1500,79.41,17,0,1.07,MX,1715528249
425,425,cushing,35.9851,-96.7670,73.51,53,0,4.61,US,1715528249
540,540,newman,37.3138,-121.0208,74.01,32,0,1.36,US,1715528175
561,561,fort irwin,35.2627,-116.6848,71.13,24,0,0.00,US,1715528259
571,571,sciacca,37.5082,13.0881,79.03,64,0,4.43,IT,1715528260


### Step 3: Create a new DataFrame called `hotel_df`.

In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Lat,Lng,Humidity,Hotel Name
32,villa de arista,MX,22.6500,-100.8333,23,
40,hamilton,US,39.1834,-84.5333,46,
73,greenville,US,35.6127,-77.3663,40,
184,bloemfontein,ZA,-29.1211,26.2140,12,
364,covington,US,39.0837,-84.5086,47,
424,heroica caborca,MX,30.7167,-112.1500,17,
425,cushing,US,35.9851,-96.7670,53,
540,newman,US,37.3138,-121.0208,32,
561,fort irwin,US,35.2627,-116.6848,24,
571,sciacca,IT,37.5082,13.0881,64,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [12]:
# Set parameters to search for a hotel
radius = 10000
params = {
    'categories': 'accommodation.hotel',  
    'limit': 1,  
    'apiKey': geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row['Lat'] 
    lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
villa de arista - nearest hotel: No hotel found
hamilton - nearest hotel: No hotel found
greenville - nearest hotel: No hotel found
bloemfontein - nearest hotel: No hotel found
covington - nearest hotel: No hotel found
heroica caborca - nearest hotel: No hotel found
cushing - nearest hotel: No hotel found
newman - nearest hotel: No hotel found
fort irwin - nearest hotel: No hotel found
sciacca - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
32,villa de arista,MX,22.6500,-100.8333,23,No hotel found
40,hamilton,US,39.1834,-84.5333,46,No hotel found
73,greenville,US,35.6127,-77.3663,40,No hotel found
184,bloemfontein,ZA,-29.1211,26.2140,12,No hotel found
364,covington,US,39.0837,-84.5086,47,No hotel found
424,heroica caborca,MX,30.7167,-112.1500,17,No hotel found
425,cushing,US,35.9851,-96.7670,53,No hotel found
540,newman,US,37.3138,-121.0208,32,No hotel found
561,fort irwin,US,35.2627,-116.6848,24,No hotel found
571,sciacca,IT,37.5082,13.0881,64,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [13]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points('Lng', 'Lat', geo=True, size='Humidity',
                                  hover_cols=['City', 'Country', 'Hotel Name', 'Humidity'],
                                  color='Humidity', cmap='blues', 
                                  tiles='OSM', frame_width=700, frame_height=500,
                                  title='City Weather and Hotels')


# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity,City,Country,Hotel Name)